# Grundlegende Infos

**Aufgabe:**
Einlesen von Ergebnissdateien im LENEX Format zur vereinfachten Auswertung der Sachsenliga

**Eingabedateien:** LEF-Dateien

# Notwendige Imports & Einstellungen & Vorbereitung

## Imports

In [7]:
import pandas as pd
from lxml import etree
from datetime import datetime, timedelta, date

## Einstellungen

In [2]:
input_file = "../data/230401-Zwickau-PR.lef"
region_code_sax = 20

## Benötigte Klassen

In [34]:
class Athlete:

    def __init__(self, name, first_name, club, gender, birth_date):
        self.name = name
        self.first_name = first_name
        self.club = club
        self.birth_data = datetime.strptime(birth_date, "%Y-%m-%d")
        self.gender = gender
        self.results = {}

    def get_birth_year(self):
        pass

class Result:

    def __init__(self, event_id, swim_time):
        self.event_id = int(event_id)
        # Parse swim time
        t = datetime.strptime(swim_time,"%H:%M:%S.%f")
        self.time = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second, microseconds=t.microsecond)
        self.points = 0

## Einlesen der Datei

In [4]:
tree = etree.parse(input_file)
root = tree.getroot()

## Übersicht über alle relevanten Vereine
Darstellung aller Clubs, welche aus Sachsen kommen

In [5]:
headers = ["Name"]
rows = []
for club in root.xpath("//CLUB"):
    if int(club.get("region")) != region_code_sax:
        continue
    rows.append([club.get("name"), ])

df_clubs = pd.DataFrame(rows, columns=headers)
df_clubs

,Name
0,1. Chemnitzer Tauchverein e.V.
1,Landestauchsportverband Sachsen
2,SC DHfK Leipzig Flossenschwimmen
3,SC Riesa Sekt. Flossenschwimmen
4,SG Dresden
5,Tauchclub NEMO Plauen e.V.
6,Tauchsport Döbeln
7,TC Delitzsch


# Einlesen der angebotenen Strecken

In [6]:
headers = ["ID", "Gender", "Style", "Distance"]
rows = []
for event in root.xpath("//EVENT"):
    style = event.xpath("SWIMSTYLE")[0]

    #Kick out relays
    if int(style.get("relaycount")) > 1:
        continue

    rows.append([event.get("eventid"), event.get("gender"), style.get("stroke"), style.get("distance")])

df_events = pd.DataFrame(rows, columns=headers)
df_events = df_events.set_index(["ID",])
df_events

,Gender,Style,Distance
ID,,,
1,F,SURFACE,400
2,M,SURFACE,400
3,F,IMMERSION,50
4,M,IMMERSION,50
5,F,APNEA,50
6,M,APNEA,50
7,F,SURFACE,100
8,M,SURFACE,100
9,F,SURFACE,800


# Einlesen der relevanten Athleten + Ergebnisse

In [33]:
athletes = []

for athlete_obj in root.xpath("//ATHLETE"):
    # Kick out athletes which are not part of the right club
    club_region_id = athlete_obj.xpath('./ancestor::CLUB/@region')

    if int(club_region_id[0]) != region_code_sax:
        continue

    # Create the athlete
    club = athlete_obj.xpath('./ancestor::CLUB/@name')[0]
    athlete = Athlete(athlete_obj.get("lastname"), athlete_obj.get("firstname"), club, athlete_obj.get("gender"), athlete_obj.get("birthdate"))


    # Read the results
    for result_obj in athlete_obj.xpath("RESULTS/RESULT"):
        event_id = int(result_obj.get("eventid"))
        result = Result(event_id, result_obj.get("swimtime"))

        athlete.results[event_id] = result

    # Add the finished athlete object to the list
    athletes.append(athlete)

# Ausgabe Ergebnis
print("Es wurden %d Athleten eingelesen" % len(athletes))

Es wurden 115 Athleten eingelesen


# Auswerten nach Events